In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [3]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [10]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///""C:/Users/nigan/sqlalchemy-challenge/resources/hawaii.sqlite")

In [11]:
from sqlalchemy import inspect
inspector = inspect(engine)
table_names = inspector.get_table_names()
table_names

['measurement', 'station']

In [13]:
# reflect an existing database into a new model
base = automap_base()

# reflect the tables
base.prepare(engine, reflect=True)

In [14]:
# View all of the classes that automap found
print(base.classes.keys())

['measurement', 'station']


In [19]:
# Save references to each table
Measurement = base.classes.measurement
Station = base.classes.station

In [20]:
# import the Session class
from sqlalchemy.orm import Session

# Create our session (link) from Python to the DB
session = Session(bind=engine)

### Test Query 

In [22]:
# engine = create_engine("sqlite:///C:/Users/nigan/sqlalchemy-challenge/resources/hawaii.sqlite")
# session = Session(bind=engine)

# # query all rows from the Measurement table
# results = session.query(Station).all()

# # print the results
# for row in results:
#     print(row)

# Exploratory Precipitation Analysis

In [ ]:
# Find the most recent date in the data set.


In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 

# Calculate the date one year from the last date in data set.


# Perform a query to retrieve the data and precipitation scores


# Save the query results as a Pandas DataFrame. Explicitly set the column names


# Sort the dataframe by date


# Use Pandas Plotting with Matplotlib to plot the data


In [ ]:
# Use Pandas to calculate the summary statistics for the precipitation data


# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number of stations in the dataset


In [ ]:
# Design a query to find the most active stations (i.e. which stations have the most rows?)
# List the stations and their counts in descending order.


In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.


In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


# Close Session

In [ ]:
# Close Session
session.close()